In [28]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:130% !important; }</style>"))

In [ ]:
# <table class="tfo-notebook-buttons" align="left">
#   <td>
#     <a target="_blank" href="https://github.com/philhoonoh/blog_git/blob/main/Dataset.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View Source</a>
#   </td>
# </table>

# PyTorch Dataset 심화 part1)

> 앞서 배운 Pytorch Dataset 의 예시코드를 분석합니다.   
> 해당 코드는 [https://boostcamp.connect.or.kr/] 에서 참조했음을 알려드립니다. 

### data 설명
> age, gender, mask 을 구별하는 모델이 대한 Dataset Class 입니다.  
> age, gender, mask 의 클래스는 각각 3, 2, 3 개 입니다.  


In [11]:
import os 
import random
from collections import defaultdict
from typing import Tuple, List

from enum import Enum

from PIL import Image
from torch.utils.data import Dataset, Subset, random_split
from torchvision import transforms
from torchvision.transforms import *
import numpy as np

IMG_EXTENSIONS = [
    ".jpg", ".JPG", ".jpeg", ".JPEG", ".png",
    ".PNG", ".ppm", ".PPM", ".bmp", ".BMP",
]

In [12]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

### torchvision.transforms
> BaseAugmentation : data augmentation 을 위한 transforms.Compose 를 Class로 구현되 있습니다.   
> Add Gaussian 에서 볼수 있듯이 직접 transorm 에 사용할 수 있는 Class 를 구현이 가능합니다!! 

In [15]:
class BaseAugmentation:
    def __init__(self, resize, mean, std, **args):
        self.transform = transforms.Compose([
            Resize(resize, Image.BILINEAR),
            ToTensor(),
            Normalize(mean=mean, std=std),
        ])

    def __call__(self, image):
        return self.transform(image)

In [16]:
class AddGaussianNoise(object):
    """
        transform 에 없는 기능들은 이런식으로 __init__, __call__, __repr__ 부분을
        직접 구현하여 사용할 수 있습니다.
    """

    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

> CustomAugmentation : data augmentation 을 위한 transforms.Compose 를 Class로 구현되 있습니다. 

In [19]:
class CustomAugmentation:
    def __init__(self, resize, mean, std, **args):
        self.transform = transforms.Compose([
            CenterCrop((320, 256)),
            Resize(resize, Image.BILINEAR),
            ColorJitter(0.1, 0.1, 0.1, 0.1),
            ToTensor(),
            Normalize(mean=mean, std=std),
            AddGaussianNoise()
        ])

    def __call__(self, image):
        return self.transform(image)

### enum.Enum Class 
> MaskLabels : enum.Enum Class로 구현되 있습니다.   
> GenderLabels : enum.Enum Class로 구현되 있습니다.   
> AgeLabels : enum.Enum Class로 구현되 있습니다.  

> 처음에 여기까지 구현할 필요가 있어나 생각했으나, 뒤의 코드를 참고시 매우 잘 만들어진 클래스라는 것을 알수 있었습니다. 

#### enum.Enum Class Usuage
- Enum is a class in python for creating enumerations, which are a set of symbolic names (members) bound to unique, constant values. The members of an enumeration can be compared by these symbolic anmes, and the enumeration itself can be iterated over. An enum has the following characteristics.
    - The enums are evaluatable string representation of an object also called repr().  
    - The name of the enum is displayed using ‘name’ keyword.  
    - Using type() we can check the enum types.  

```python
import enum
# Using enum class create enumerations
class Days(enum.Enum):
   Sun = 1
   Mon = 2
   Tue = 3

# print the enum member as a string
print ("The enum member as a string is : ",end="")
print (Days.Mon)

# print the enum member as a repr
print ("he enum member as a repr is : ",end="")
print (repr(Days.Sun))

# Check type of enum member
print ("The type of enum member is : ",end ="")
print (type(Days.Mon))

# print name of enum member
print ("The name of enum member is : ",end ="")
print (Days.Tue.name)
```

In [21]:
class MaskLabels(int, Enum):
    MASK = 0
    INCORRECT = 1
    NORMAL = 2

In [33]:
print(type(MaskLabels))
print(MaskLabels.MASK)
print(MaskLabels.INCORRECT)
print(MaskLabels.NORMAL)

<class 'enum.EnumMeta'>
MaskLabels.MASK
MaskLabels.INCORRECT
MaskLabels.NORMAL
MaskLabels.MASK


In [26]:
class GenderLabels(int, Enum):
    MALE = 0
    FEMALE = 1

    @classmethod
    def from_str(cls, value: str) -> int:
        value = value.lower()
        if value == "male":
            return cls.MALE
        elif value == "female":
            return cls.FEMALE
        else:
            raise ValueError(f"Gender value should be either 'male' or 'female', {value}")


In [39]:
gender = GenderLabels.from_str('male')
gender * 5

0

In [27]:
class AgeLabels(int, Enum):
    YOUNG = 0
    MIDDLE = 1
    OLD = 2

    @classmethod
    def from_number(cls, value: str) -> int:
        try:
            value = int(value)
        except Exception:
            raise ValueError(f"Age value should be numeric, {value}")

        if value < 30:
            return cls.YOUNG
        elif value < 60:
            return cls.MIDDLE
        else:
            return cls.OLD

In [38]:
age = AgeLabels.from_number(40)
age * 43

43

### MaskBaseDataset
> 분석할 Dataset Class 입니다. 
> 매우 길어 하나씩 분석해 보겠습니다. 

In [30]:
class MaskBaseDataset(Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, data_dir, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), val_ratio=0.2):
        self.data_dir = data_dir
        self.mean = mean
        self.std = std
        self.val_ratio = val_ratio

        self.transform = None
        self.setup()
        self.calc_statistics()

    def setup(self):
        profiles = os.listdir(self.data_dir)
        for profile in profiles:
            if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue

            img_folder = os.path.join(self.data_dir, profile)
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                    continue

                img_path = os.path.join(self.data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                mask_label = self._file_names[_file_name]

                id, gender, race, age = profile.split("_")
                gender_label = GenderLabels.from_str(gender)
                age_label = AgeLabels.from_number(age)

                self.image_paths.append(img_path)
                self.mask_labels.append(mask_label)
                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)

    def calc_statistics(self):
        has_statistics = self.mean is not None and self.std is not None
        if not has_statistics:
            print("[Warning] Calculating statistics... It can take a long time depending on your CPU machine")
            sums = []
            squared = []
            for image_path in self.image_paths[:3000]:
                image = np.array(Image.open(image_path)).astype(np.int32)
                sums.append(image.mean(axis=(0, 1)))
                squared.append((image ** 2).mean(axis=(0, 1)))

            self.mean = np.mean(sums, axis=0) / 255
            self.std = (np.mean(squared, axis=0) - self.mean ** 2) ** 0.5 / 255

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, index):
        assert self.transform is not None, ".set_tranform 메소드를 이용하여 transform 을 주입해주세요"

        image = self.read_image(index)
        mask_label = self.get_mask_label(index)
        gender_label = self.get_gender_label(index)
        age_label = self.get_age_label(index)
        multi_class_label = self.encode_multi_class(mask_label, gender_label, age_label)

        image_transform = self.transform(image)
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

    def get_mask_label(self, index) -> MaskLabels:
        return self.mask_labels[index]

    def get_gender_label(self, index) -> GenderLabels:
        return self.gender_labels[index]

    def get_age_label(self, index) -> AgeLabels:
        return self.age_labels[index]

    def read_image(self, index):
        image_path = self.image_paths[index]
        return Image.open(image_path)

    @staticmethod
    def encode_multi_class(mask_label, gender_label, age_label) -> int:
        return mask_label * 6 + gender_label * 3 + age_label

    @staticmethod
    def decode_multi_class(multi_class_label) -> Tuple[MaskLabels, GenderLabels, AgeLabels]:
        mask_label = (multi_class_label // 6) % 3
        gender_label = (multi_class_label // 3) % 2
        age_label = multi_class_label % 3
        return mask_label, gender_label, age_label

    @staticmethod
    def denormalize_image(image, mean, std):
        img_cp = image.copy()
        img_cp *= std
        img_cp += mean
        img_cp *= 255.0
        img_cp = np.clip(img_cp, 0, 255).astype(np.uint8)
        return img_cp

    def split_dataset(self) -> Tuple[Subset, Subset]:
        """
        데이터셋을 train 과 val 로 나눕니다,
        pytorch 내부의 torch.utils.data.random_split 함수를 사용하여
        torch.utils.data.Subset 클래스 둘로 나눕니다.
        구현이 어렵지 않으니 구글링 혹은 IDE (e.g. pycharm) 의 navigation 기능을 통해 코드를 한 번 읽어보는 것을 추천드립니다^^
        """
        n_val = int(len(self) * self.val_ratio)
        n_train = len(self) - n_val
        train_set, val_set = random_split(self, [n_train, n_val])
        return train_set, val_set

#### step1
> num_classes, _file_names 등 클래스 개수 및 labeling dictionary를 설정합니다.  
> image_paths, mask_labels, gender_labels, age_labels 등 map-style dataset 의 인덱싱을 위해 설정합니다. 


In [125]:
num_classes = 3 * 2 * 3

_file_names = {
    "mask1": MaskLabels.MASK,
    "mask2": MaskLabels.MASK,
    "mask3": MaskLabels.MASK,
    "mask4": MaskLabels.MASK,
    "mask5": MaskLabels.MASK,
    "incorrect_mask": MaskLabels.INCORRECT,
    "normal": MaskLabels.NORMAL
}

image_paths = []
mask_labels = []
gender_labels = []
age_labels = []

data_dir = '/opt/ml/input/data/train/images'
profiles = os.listdir(data_dir)

#### step2 - setup
> 해당 부분은 난해애 보이나 data가 어떻게 저장되어 있는지에 따라 경로를 찾는 과정일 뿐입니다.  
> 위에서 정의한 image_paths, mask_labels, gender_labels, age_labels 에 사진 정보를 저장합니다.  

In [138]:
for ind, profile in enumerate(profiles):
    if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue
    
    img_folder = os.path.join(data_dir, profile)
    print(f'ind : {ind}, profile : {profile}, img_folder : {img_folder}')
    for file_name in os.listdir(img_folder):
        _file_name, ext = os.path.splitext(file_name)
        if _file_name not in _file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
            continue
        
        img_path = os.path.join(data_dir, profile, file_name)
        
        mask_label = _file_names[_file_name]
        print(f'img_path " {img_path}')
        print(f'mask_label " {mask_label}')
        id, gender, race, age = profile.split("_")
        gender_label = GenderLabels.from_str(gender)
        age_label = AgeLabels.from_number(age)
        print(f'gender_label " {gender_label}')
        print(f'age_label " {age_label}')
        

ind : 0, profile : 000279_female_Asian_54, img_folder : /opt/ml/input/data/train/images/000279_female_Asian_54
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/normal.jpg
mask_label " 2
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/mask1.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000279_female_Asian_54/mask5.jpg
mask_label " 0
gender_label " 1
age_label " 1
ind : 2, profile : 004219

gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/006259_female_Asian_20/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/006259_female_Asian_20/normal.jpg
mask_label " 2
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/006259_female_Asian_20/mask1.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/006259_female_Asian_20/mask5.jpg
mask_label " 0
gender_label " 1
age_label " 0
ind : 99, profile : 001795_male_Asian_27, img_folder : /opt/ml/input/data/train/images/001795_male_Asian_27
img_path " /opt/ml/input/data/train/images/001795_male_Asian_27/incorrect_mask.jpg
mask_label " 1
gender_label " 0
age_label " 0
img_path " /opt/ml/input/data/train/images/001795_male_Asian_27/mask2.jpg
mask_label " 0
gender_label " 0
age_label " 0
img_path " /opt/ml/input/data/train/images/001795_male_Asian_27/mask4.jpg
mask_label " 0
gender_label " 0
age_label " 0
img

mask_label " 0
gender_label " 0
age_label " 0
img_path " /opt/ml/input/data/train/images/001815_male_Asian_26/mask5.jpg
mask_label " 0
gender_label " 0
age_label " 0
ind : 157, profile : 001158_female_Asian_56, img_folder : /opt/ml/input/data/train/images/001158_female_Asian_56
img_path " /opt/ml/input/data/train/images/001158_female_Asian_56/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001158_female_Asian_56/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001158_female_Asian_56/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001158_female_Asian_56/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001158_female_Asian_56/normal.jpg
mask_label " 2
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001158_female_Asian_56/mask1.jpg
mask_label " 0
gender_lab

age_label " 1
img_path " /opt/ml/input/data/train/images/000661_male_Asian_55/normal.jpg
mask_label " 2
gender_label " 0
age_label " 1
img_path " /opt/ml/input/data/train/images/000661_male_Asian_55/mask1.jpg
mask_label " 0
gender_label " 0
age_label " 1
img_path " /opt/ml/input/data/train/images/000661_male_Asian_55/mask5.jpg
mask_label " 0
gender_label " 0
age_label " 1
ind : 259, profile : 001825_female_Asian_21, img_folder : /opt/ml/input/data/train/images/001825_female_Asian_21
img_path " /opt/ml/input/data/train/images/001825_female_Asian_21/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001825_female_Asian_21/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001825_female_Asian_21/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001825_female_Asian_21/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt

age_label " 1
img_path " /opt/ml/input/data/train/images/004313_female_Asian_56/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/004313_female_Asian_56/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/004313_female_Asian_56/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/004313_female_Asian_56/normal.jpg
mask_label " 2
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/004313_female_Asian_56/mask1.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/004313_female_Asian_56/mask5.jpg
mask_label " 0
gender_label " 1
age_label " 1
ind : 334, profile : 004247_female_Asian_50, img_folder : /opt/ml/input/data/train/images/004247_female_Asian_50
img_path " /opt/ml/input/data/train/images/004247_female_Asian_50/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 1
img_path 

...

img_path " /opt/ml/input/data/train/images/006959_male_Asian_19/normal.jpg
mask_label " 2
gender_label " 0
age_label " 0
img_path " /opt/ml/input/data/train/images/006959_male_Asian_19/mask1.jpg
mask_label " 0
gender_label " 0
age_label " 0
img_path " /opt/ml/input/data/train/images/006959_male_Asian_19/mask5.jpg
mask_label " 0
gender_label " 0
age_label " 0
ind : 4414, profile : 000212_female_Asian_51, img_folder : /opt/ml/input/data/train/images/000212_female_Asian_51
img_path " /opt/ml/input/data/train/images/000212_female_Asian_51/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000212_female_Asian_51/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000212_female_Asian_51/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/000212_female_Asian_51/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/dat

img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/incorrect_mask.jpg
mask_label " 1
gender_label " 0
age_label " 2
img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/mask2.jpg
mask_label " 0
gender_label " 0
age_label " 2
img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/mask4.jpg
mask_label " 0
gender_label " 0
age_label " 2
img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/mask3.jpg
mask_label " 0
gender_label " 0
age_label " 2
img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/normal.jpg
mask_label " 2
gender_label " 0
age_label " 2
img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/mask1.jpg
mask_label " 0
gender_label " 0
age_label " 2
img_path " /opt/ml/input/data/train/images/004385_male_Asian_60/mask5.jpg
mask_label " 0
gender_label " 0
age_label " 2
ind : 4603, profile : 001033_female_Asian_19, img_folder : /opt/ml/input/data/train/images/001033_female_Asian_19
img_path " /opt/ml/input/data/train/

gender_label " 0
age_label " 1
img_path " /opt/ml/input/data/train/images/001090_male_Asian_34/normal.jpg
mask_label " 2
gender_label " 0
age_label " 1
img_path " /opt/ml/input/data/train/images/001090_male_Asian_34/mask1.jpg
mask_label " 0
gender_label " 0
age_label " 1
img_path " /opt/ml/input/data/train/images/001090_male_Asian_34/mask5.jpg
mask_label " 0
gender_label " 0
age_label " 1
ind : 4834, profile : 004392_female_Asian_18, img_folder : /opt/ml/input/data/train/images/004392_female_Asian_18
img_path " /opt/ml/input/data/train/images/004392_female_Asian_18/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/004392_female_Asian_18/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/004392_female_Asian_18/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/004392_female_Asian_18/mask3.jpg
mask_label " 0
gender_label " 1
age_label "

img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/normal.jpg
mask_label " 2
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/mask1.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/001289_female_Asian_23/mask5.jpg
mask_label " 0
gender_label " 1
age_label " 0
ind : 5029, profile : 005011_female_Asian_45, img_folder : /opt/ml/input/data/train/images/005011_female_Asian_45
img_path " /opt/ml/inp

img_path " /opt/ml/input/data/train/images/001083_female_Asian_34/mask4.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001083_female_Asian_34/mask3.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001083_female_Asian_34/normal.jpg
mask_label " 2
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001083_female_Asian_34/mask1.jpg
mask_label " 0
gender_label " 1
age_label " 1
img_path " /opt/ml/input/data/train/images/001083_female_Asian_34/mask5.jpg
mask_label " 0
gender_label " 1
age_label " 1
ind : 5257, profile : 003183_female_Asian_19, img_folder : /opt/ml/input/data/train/images/003183_female_Asian_19
img_path " /opt/ml/input/data/train/images/003183_female_Asian_19/incorrect_mask.jpg
mask_label " 1
gender_label " 1
age_label " 0
img_path " /opt/ml/input/data/train/images/003183_female_Asian_19/mask2.jpg
mask_label " 0
gender_label " 1
age_label " 0
img_path " /opt/ml/inp

#### step3 - calc_statistics
> 이미지의 statistics 를 계산합니다. 

In [168]:
sums = []

In [174]:
image_path = '/opt/ml/input/data/train/images/004219_male_Asian_60/mask2.jpg'
image_path = '/opt/ml/input/data/train/images/004219_male_Asian_60/mask4.jpg'

In [175]:
image = np.array(Image.open(image_path)).astype(np.int32)
image.shape

(512, 384, 3)

In [176]:
means = image.mean(axis=(0, 1))

In [177]:
means.shape

(3,)

In [178]:
sums.append(means)

In [179]:
sums

[array([151.26723735, 125.40224202, 118.94506327]),
 array([151.26723735, 125.40224202, 118.94506327])]

In [180]:
np.mean(sums, axis=0)

array([151.26723735, 125.40224202, 118.94506327])

#### step4 나머지 함수들은 이해하기 어렵지 않습니다만 이후 part2) 에 업로드하겠습니다. 

In [121]:
class MaskBaseDataset(Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, data_dir, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), val_ratio=0.2):
        self.data_dir = data_dir
        self.mean = mean
        self.std = std
        self.val_ratio = val_ratio

        self.transform = None
        self.setup()
        self.calc_statistics()

    def setup(self):
        profiles = os.listdir(self.data_dir)
        for profile in profiles:
            if profile.startswith("."):  # "." 로 시작하는 파일은 무시합니다
                continue

            img_folder = os.path.join(self.data_dir, profile)
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names:  # "." 로 시작하는 파일 및 invalid 한 파일들은 무시합니다
                    continue

                img_path = os.path.join(self.data_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                mask_label = self._file_names[_file_name]

                id, gender, race, age = profile.split("_")
                gender_label = GenderLabels.from_str(gender)
                age_label = AgeLabels.from_number(age)

                self.image_paths.append(img_path)
                self.mask_labels.append(mask_label)
                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)

    def calc_statistics(self):
        has_statistics = self.mean is not None and self.std is not None
        if not has_statistics:
            print("[Warning] Calculating statistics... It can take a long time depending on your CPU machine")
            sums = []
            squared = []
            for image_path in self.image_paths[:3000]:
                image = np.array(Image.open(image_path)).astype(np.int32)
                sums.append(image.mean(axis=(0, 1)))
                squared.append((image ** 2).mean(axis=(0, 1)))

            self.mean = np.mean(sums, axis=0) / 255
            self.std = (np.mean(squared, axis=0) - self.mean ** 2) ** 0.5 / 255

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, index):
        assert self.transform is not None, ".set_tranform 메소드를 이용하여 transform 을 주입해주세요"

        image = self.read_image(index)
        mask_label = self.get_mask_label(index)
        gender_label = self.get_gender_label(index)
        age_label = self.get_age_label(index)
        multi_class_label = self.encode_multi_class(mask_label, gender_label, age_label)

        image_transform = self.transform(image)
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

    def get_mask_label(self, index) -> MaskLabels:
        return self.mask_labels[index]

    def get_gender_label(self, index) -> GenderLabels:
        return self.gender_labels[index]

    def get_age_label(self, index) -> AgeLabels:
        return self.age_labels[index]

    def read_image(self, index):
        image_path = self.image_paths[index]
        return Image.open(image_path)

    @staticmethod
    def encode_multi_class(mask_label, gender_label, age_label) -> int:
        return mask_label * 6 + gender_label * 3 + age_label

    @staticmethod
    def decode_multi_class(multi_class_label) -> Tuple[MaskLabels, GenderLabels, AgeLabels]:
        mask_label = (multi_class_label // 6) % 3
        gender_label = (multi_class_label // 3) % 2
        age_label = multi_class_label % 3
        return mask_label, gender_label, age_label

    @staticmethod
    def denormalize_image(image, mean, std):
        img_cp = image.copy()
        img_cp *= std
        img_cp += mean
        img_cp *= 255.0
        img_cp = np.clip(img_cp, 0, 255).astype(np.uint8)
        return img_cp

    def split_dataset(self) -> Tuple[Subset, Subset]:
        """
        데이터셋을 train 과 val 로 나눕니다,
        pytorch 내부의 torch.utils.data.random_split 함수를 사용하여
        torch.utils.data.Subset 클래스 둘로 나눕니다.
        구현이 어렵지 않으니 구글링 혹은 IDE (e.g. pycharm) 의 navigation 기능을 통해 코드를 한 번 읽어보는 것을 추천드립니다^^
        """
        n_val = int(len(self) * self.val_ratio)
        n_train = len(self) - n_val
        train_set, val_set = random_split(self, [n_train, n_val])
        return train_set, val_set

In [123]:
a = MaskBaseDataset('/opt/ml/input/data/train/images')